## Software Design
- **Recognizing constraints**: we probably want sub 100ms latency for the similarity retrieval, there is an order of magnitude 10M listings
- **Database**: always provide schemas for each database, and mention what is the primary key / partition keys / clustering keys. Partition key defines which node the data is on, clustering key defines what the data is ordered by

## Recommender Model
- **Consider KNN**: Inference is O(N*d), with N being the number of training data in KNN and d being the dimension of the embedding in each data. This is too slow. We can store the top K neighbors for each listing in a separate database, but it would need to be updated when new listings are added. This is suboptimal.
- **Consider Clustering**: Pre-compute the centroids of clusters; during inference time, compute similarities with items within the cluster. Cluster size is crutial -> if there are k items in each cluster, the inference runtime would be O(k*d) with d being the dimension of the vectors. When a new item is added, we compare it with the centroids to classify it into a cluster. Every X period of time, we redo the clustering and re-compute the centroids. If we store similar listings to a particular listing in a DB, it also would need to be updated, which wouldn't work very well.
    - K-Means: require knowing the number clusers, globular clusters, simple and fast
    - DBSCAN: arbitrary number of clusters with arbitrary shape, doesn't provide centroids but you can compute post-hoc

- **Create Embedding Model via Triplet Loss**: Create a model that maps similar listings close to each other in embedding space. Eval

In [55]:
import numpy as np
import torch

def generate_example_data(num_samples, input_dim):
    anchors = np.random.rand(num_samples, input_dim)
    positives = anchors + 0.1 * np.random.randn(num_samples, input_dim)
    negatives = np.random.rand(num_samples, input_dim)
    return anchors, positives, negatives

In [54]:
anchors, positives, negatives = generate_example_data(10, 10)
anchors.dtype

torch.float32

In [18]:
# define triplet loss function using pytorch: https://pytorch.org/docs/stable/generated/torch.nn.TripletMarginLoss.html
import torch
import torch.nn as nn
triplet_loss_fn = nn.TripletMarginLoss()

# example
anchor = torch.randn(4, requires_grad=True)
positive = torch.randn(4, requires_grad=True)
negative = torch.randn(4, requires_grad=True)
output = triplet_loss_fn(anchor, positive, negative)
output.backward() # need requires_grad for all tensors, results gets

print(f"anchor.grad = {anchor.grad}")
print(f"positive.grad = {positive.grad}")
print(f"negative.grad = {negative.grad}")

anchor.grad = tensor([-0.5567,  0.4102, -0.3252, -0.9785])
positive.grad = tensor([0.9110, 0.1212, 0.3342, 0.2091])
negative.grad = tensor([-0.3543, -0.5314, -0.0090,  0.7694])


Define an MLP as the embedding model. The output size would be the dimension of the embeddings that's genreated. This model takes in some input vector with dimension = `input_size` and outputs an embedding with dimension = `output_size`. The hidden layers would have a dimension in the `hidden_sizes`

In [30]:
class MLP(nn.Module):
    def __init__(self, input_size: int, hidden_sizes: list[int], output_size: int):
        super().__init__()

        layers = []
        prev_size = input_size

        # constructs the layers
        # Remember, all the common modules / layers support batch computation automatically
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, output_size))

        # wrap it in nn.Sequential which treats it as a single module, and provides the forward function that passes through all layers
        # https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html
        # it takes each layer as its own arguments so you have to unwrap it with *layers
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)



In [33]:
# test example
mlp = MLP(1024, [512, 256, 128, 64], 32)
input = torch.randn(1024)
out = mlp(input)
out.shape

torch.Size([32])

In [37]:
# create the dataset class. We don't need a custom collate function because the default collate function stacks the tensors, and that's what we want
from torch.utils.data import Dataset, DataLoader

class TripletDataset(Dataset):
    """
    Every custom dataset must implement __init__, __len__, and __getitem__
    """
    def __init__(self, anchors, positives, negatives):
        self.anchors = anchors
        self.positives = positives
        self.negatives = negatives

    def __len__(self):
        assert len(self.anchors) == len(self.positives)
        assert len(self.anchors) == len(self.negatives)
        return len(self.anchors)
    
    def __getitem__(self, idx):
        return self.anchors[idx], self.positives[idx], self.negatives[idx]

In [62]:
feature_dim = 1024
embedding_dim = 32
anchors, positives, negatives = generate_example_data(1000, feature_dim)

# convert them to float 32 because model weights are by default float 32. Otherwise, cannot do computations of different dtypes
anchors = torch.from_numpy(anchors).float()
positives = torch.from_numpy(positives).float()
negatives = torch.from_numpy(negatives).float()

dataset = TripletDataset(anchors, positives, negatives)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
mlp = MLP(feature_dim, [512, 256, 128, 64], embedding_dim)
triplet_loss_fn = nn.TripletMarginLoss()
optimizer = torch.optim.Adam(params=mlp.parameters(), lr=1e-4) # adam optimizer is optimizing over the model parameters, so you must pass in params

def train(num_epochs):
    for epoch in range(num_epochs):
        total_loss = 0
        
        for batch_anchors, batch_positives, batch_negatives in dataloader:
            optimizer.zero_grad() # always necessary at each iteration
            anchor_embeddings, positive_embeddings, negative_embeddings = mlp(batch_anchors), mlp(batch_positives), mlp(batch_negatives)
            loss = triplet_loss_fn(anchor_embeddings, positive_embeddings, negative_embeddings)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() # .item() gets the scalar from a tensor

        print(f"Epoch {epoch}: total loss = {total_loss}")


In [63]:
train(5)

Epoch 0: total loss = 30.614742279052734
Epoch 1: total loss = 28.332452535629272
Epoch 2: total loss = 20.422190502285957
Epoch 3: total loss = 7.288619086146355
Epoch 4: total loss = 2.1929321084171534
